In [1]:
import pymongo
import requests
import json
import time
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import sys
sys.path.append("../common")
from api_keys_local_SW import nyt_API_key

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.gs_search_db
collection = db.gs_articles

In [4]:
#define url/search params

# set up utilities/params
api_key = nyt_API_key
search_terms = ["gamestop", "reddit"]
#search_term_l = ["gamestop", "term2", "term3", "term4", "term5"]
begin_date = "20201201"
end_date = "20210220"
current_page_counter = 0
page_count_limit = 1
searchCompleteVar = False
search_term_count = 1




        url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={search_term}&begin_date={begin_date}&end_date={end_date}&page={current_page_counter}&api-key={api_key}'
print(url)


response = requests.get(url)
result = json.loads(response.content)

with open('test/articlesearch_gamestop_202102.json', encoding="UTF-8") as j:
  json_data = json.load(j)

if json_data != []:
    print("OK!")
else:
    print("NG!")

In [6]:

for term in search_terms:
    search_post = {
        }
    articles_list = []

    for p in term:
        # sleep to follow API rules
        time.sleep(6)
        # defining url
        url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?q={term}&begin_date={begin_date}&end_date={end_date}&page={current_page_counter}&api-key={api_key}'

        # set up exception handling for the call
        try:
            # make request and turn into json object
            requestObject = requests.get(url)
            response = json.loads(requestObject.text)
            print()
            print("The status code is {}".format(requestObject.status_code) + " for page " + str(current_page_counter) + " of: " + term)
            print(requestObject.url[0:-41])
        except:
            # on call error, break from current search term loop and move to next search term
            print()
            print("Invalid results. Moving to next request.")            
            current_page_counter = 0
            break

        if requestObject.status_code == 200:
            if response["response"]["docs"] != []:
            #for each article's docs section
                print(f'Getting responses...')
                for art in response["response"]["docs"]:
                    articles_list.append(art)
            else:
                print(f'End of results. Starting new search term query')
                current_page_counter = 0
                break                  
            # check if requests are at their limit for search term
            if current_page_counter == page_count_limit:
                searchCompleteVar = False
                print(f'At final page allowed.')
                current_page_counter = 0
                break
            current_page_counter += 1
    #add search term
    search_post["search_term"] = term
    #add articles list
    search_post["articles"] = articles_list
    #get hits number
    search_post["hits"] = response["response"]["meta"]["hits"]
    #add to collection
    collection.insert_one(search_post)

    #prep for next search term
    search_term_count += 1
    #move to next search term
    #add if clause for 
    print(f'Moving to search term {search_term_count}.')


The status code is 200 for page 0 of: gamestop
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=gamestop&begin_date=20201201&end_date=20210220&page=0
Getting responses.

The status code is 200 for page 1 of: gamestop
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=gamestop&begin_date=20201201&end_date=20210220&page=1
Getting responses.
At final page allowed. Starting new search term query.
Moving to search term 2.

The status code is 200 for page 0 of: reddit
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=reddit&begin_date=20201201&end_date=20210220&page=0
Getting responses.

The status code is 200 for page 1 of: reddit
https://api.nytimes.com/svc/search/v2/articlesearch.json?q=reddit&begin_date=20201201&end_date=20210220&page=1
Getting responses.
At final page allowed. Starting new search term query.
Moving to search term 3.


In [ ]:


#build result object
search_post = {}


# for loop targeting each, lowest level for entire search term
# for each search term, make request and build a dictionary
    search_post = {
        "search_term": t,        
    }

    # for each page, look into "response">"docs"
    for p in search:
        #for each article, get this info
        for art in json_data["docs"]
            search_post = {
                "article": art
            }



    post = {
        
    }





In [ ]:
collection.insert_one(fake_db)

In [ ]:
post = {
    'article_name':"name here",
    'att1':'val1',
    'att2':'val2'
}


collection.insert_one(post)

In [ ]:
hits = response["response"]["meta"]["hits"]

In [ ]:
result

In [ ]:
post = {
    'article_name':"name here",
    'att1':'val1',
    'att2':'val2'
}


collection.insert_one(post)